In [1]:
import sys
import matplotlib.pyplot as plt
sys.path.append('../')
from model.optimization_model import OptimizationModel
import pyomo.environ as pyo
import pandas as pd

In [2]:
df = pd.read_csv('../data/consumo_material_preprocessed.csv')

In [3]:
df = df[df['FECHAPEDIDO'].str.contains('2023')]
df = df.reset_index(drop=True)

CODIGO_PROD = 65056
data = df[df['CODIGO_NUM'] == CODIGO_PROD]
data.reset_index(drop=True, inplace=True)
data

,FECHAPEDIDO,NUMERO,REFERENCIA,CANTIDADCOMPRA,UNIDADESCONSUMOCONTENIDAS,PRECIO,IMPORTELINEA,TIPOCOMPRA,TGL,PRODUCTO,CODIGO_CLASS,CODIGO_NUM,PURCHASING_REGION,PURCHASING_HOSPITAL,PURCHASING_DEPARTMENT
0,2023-01-03,4051/23,33533,1008,3,0.989571,332.495856,Concurso,ALMACENABLE,APOSITO DE HIDROCOLOIDE-9,E,65056,0,0,2
1,2023-01-09,16119/23,33533,216,3,0.989571,71.249112,Concurso,ALMACENABLE,APOSITO DE HIDROCOLOIDE-9,E,65056,0,18,1
2,2023-01-11,28629/23,33533,180,3,0.989571,59.374260,Concurso,ALMACENABLE,APOSITO DE HIDROCOLOIDE-9,E,65056,0,4,1
3,2023-01-13,40354/23,33533,261,3,0.989571,86.092677,Concurso,ALMACENABLE,APOSITO DE HIDROCOLOIDE-9,E,65056,0,10,1
4,2023-01-20,75411/23,33533,228,3,0.989571,75.207396,Concurso,ALMACENABLE,APOSITO DE HIDROCOLOIDE-9,E,65056,0,18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2023-09-05,1081661/23,33533,1650,3,0.989571,544.264050,Compra menor,ALMACENABLE,APOSITO DE HIDROCOLOIDE-9,E,65056,0,8,1
72,2023-09-25,1182299/23,33533,1260,3,0.989571,415.619820,Compra menor,ALMACENABLE,APOSITO DE HIDROCOLOIDE-9,E,65056,0,0,2
73,2023-10-03,1225479/23,33533,1521,3,0.989571,501.712497,Compra menor,ALMACENABLE,APOSITO DE HIDROCOLOIDE-9,E,65056,0,8,1
74,2023-10-16,1268439/23,33533,600,3,0.989571,197.914200,Compra menor,ALMACENABLE,APOSITO DE HIDROCOLOIDE-9,E,65056,0,7,3


In [4]:
model = OptimizationModel(data, 1, 4, 10000, 5)

In [5]:
solver_path = 'C:/Users/joels/Desktop/Uni/datathon 2023/winglpk-4.65/glpk-4.65/w64/glpsol.exe'
best_model = model.solve(solver_path)

if best_model is None:
    print('No solution found')
else:
    optimal_values = [pyo.value(best_model.p[i]) for i in best_model.p]
    print(f'Optimal values: {optimal_values}')

  0%|          | 0/495 [00:00<?, ?it/s]

100%|██████████| 495/495 [00:31<00:00, 15.86it/s]

No solution found


In [6]:
best_model.objective()
best_model.delta

AttributeError: 'NoneType' object has no attribute 'objective'